In [1]:
import pandas as pd
import numpy as np
import re
import pickle
import time
from pymongo import MongoClient

To add a new book to the Mongo database, follow the code below. Then, either run the cleaning and lemmatizing functions on the entire database or load a pickle file with the lemmatized texts.

In [2]:
client=MongoClient()
db = client['proj_4_gutenberg']
collection = db['gutenberg_books']

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

## Mongo single book import

In [4]:
def book_to_mongo(book_num):

    book_details = {}
    book_details['_id']  = str(book_num)
    driver = webdriver.Chrome(chromedriver)
    try:
        driver.get('https://www.gutenberg.org/ebooks/%s' % book_num)
        book_text = driver.find_element_by_link_text('Read this book online: HTML')
        #     book_text=driver.find_element_by_xpath('//*[@id="download"]/div/table/tbody/tr[8]/td[2]/a')
        book_text.click()
        time.sleep(2)
        pre_mat = driver.find_elements_by_tag_name('pre')
      
        title_search = re.compile('Title:(.*?)\\n')
        title = title_search.search(str(pre_mat[0].text))
        print('hello')
        title = title.group(0)
        
        just_title = re.compile(':(.*)')
        book_title = just_title.search(title)
        title = book_title.group(1).strip()
        print(title)
        book_details['title'] = title


        author_search = re.compile('Author:(.*?)\\n')
        author = author_search.search(str(pre_mat[0].text))
        author = str(author.group(0))
        just_name = re.compile(':(.*)')
        author_name = just_name.search(author)
        author = author_name.group(1).strip()
        book_details['author'] = author


        book_text = driver.find_elements_by_tag_name('p')
        text_for_dict = ''
        for x in range(len(book_text)):
            text_for_dict += (book_text[x].text)
        book_details['text'] = text_for_dict

        print(book_details)
        driver.close()
        
        collection.insert_one(book_details)
    except:
        driver.close()
    print(book_details)
    return book_details

In [18]:
book_to_mongo(5230)

hello
The Invisible Man


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=72.0.3626.119)
  (Driver info: chromedriver=2.35.528157 (4429ca2590d6988c0745c24c8858745aaaec01ef),platform=Mac OS X 10.14.3 x86_64)


## Lemmatize a single book and add to list/dict

To add a lemmatized book to the pickle file, run the function below.

In [5]:
# %autoreload 1
# %aimport spacy_lemmer

import sys
sys.path.insert(0, '../src/features/')
sys.path.insert(0, '../src/data/')

import spacy_lemmer
from clean_book import book_cleaner

## Add to list

In [6]:
def new_text_to_lem_pickle(book_num):
    with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
        lemmed_text = pickle.load(input_file)
    
    book_for_lemming = book_cleaner(book_num)
    spacy_lemmed_text = spacy_lemmer.spacy_lem_text(book_for_lemming)
    lemmed_text.append(spacy_lemmed_text)
    
    with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "wb") as output_file:
        pickle.dump(lemmed_text, output_file)


In [20]:
new_text_to_lem_pickle()

The Dunwich Horror 99400
88458


## Add to dictionary

In [54]:
def new_text_to_lem_pickle(book_num):
    with open('../data/processed/lem_dict_500000.pk', "rb") as input_file:
        lemmed_dict = pickle.load(input_file)
    
    book_for_lemming = book_cleaner(book_num)
    spacy_lemmed_text = spacy_lemmer.spacy_lem_text(book_for_lemming)
    book_num = str(book_num)
    lemmed_dict[book_num] = spacy_lemmed_text
    
    with open('../data/processed/lemmed_dict_500000.pk', "wb") as output_file:
        pickle.dump(lemmed_text, output_file)

Either run the full cleaning and lemmatizing code commented out below, or load the pickled lemmed_text file.  Lemmatization takes a very long time, so a pickled list of the lemmatized texts is loaded.

In [55]:
with open('../data/processed/lem_dict_500000.pk', "rb") as input_file:
    lemmed_text = pickle.load(input_file)

In [56]:
lemmed_text['50133']

'    a traveler in north central take the wrong fork at the junction of the pike just beyond \'s corners come upon a lonely and curious country . the ground get high , and the brier - border stone wall press closer and close against the rut of the dusty , curve road . the tree of the frequent forest belt seem too large , and the wild weed , bramble , and grass attain a luxuriance not often find in settled region . at the same time the plant field appear singularly few and barren ; while the sparsely scattered house wear a surpriz uniform aspect of age , squalor , and dilapidation . without know why , one hesitate to ask direction from the gnarl , solitary figure spy now and then on crumble doorstep or in the sloping , rock - strew meadow . those figure be so silent and furtive that one feel somehow confront by forbidden thing , with which would be good to have nothing to do . when a rise in the road bring the mountain in view above the deep wood , the feeling of strange uneasiness be i

In [ ]:
# books_for_lemming = [book_cleaner(str(book)) for book in book_nums]
# spacy_lemmed_text = [spacy_lemmer.spacy_lem_text(book) for book in books_for_lemming]

In [7]:
# with open('../data/processed/spacy_lemmed_train_text_500000.pickle', "rb") as input_file:
#     lemmed_text = pickle.load(input_file)

Query the database for all of the book information.  The dictionary and lists will be used for identifying the books while clustering, comparing, and building the Flask app.

In [160]:
def get_book_info():    
    book_ids = collection.find({}, {'_id':1, 'title':1, 'author':1})
    books_for_dict = {}
    book_nums = []
    book_titles = []
    book_authors = []
    for book in book_ids:
        book_nums.append((book['_id']))
        book_titles.append((book['title']))
        book_authors.append((book['author']))

        url = 'https://www.gutenberg.org/ebooks/%s' % book['_id']
        books_for_dict[book['title']]=[book['author'],url]
    
    return [books_for_dict, book_nums, book_titles, book_authors]

book_info = get_book_info()
books_for_dict, book_nums, book_titles, book_authors = (book_info[0],
                                                        book_info[1],
                                                        book_info[2],
                                                        book_info[3])

In [92]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
lemmed_text_from_dict = [lemmed_text[_id] for _id in book_nums]

## tfidf vectorizor

In [149]:
tfidf= TfidfVectorizer(min_df = .1, max_df = .8, stop_words='english', ngram_range=(1,2))
doc_word = tfidf.fit_transform(lemmed_text_from_dict)

In [150]:
# with open('../src/models/tf_idf.pk', 'wb') as write_file:
#     pickle.dump(tfidf, write_file)

## truncatedSVD 

Taked the lemmed text and performed truncatedSVD modeling on the tfidf vectorized text.

In [156]:
lsa = TruncatedSVD(45)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.02629059, 0.04742947, 0.04673746, 0.03663393, 0.02779345,
       0.02354128, 0.02211226, 0.02190526, 0.02009459, 0.01932687,
       0.01771718, 0.01725058, 0.01649899, 0.01608567, 0.01607723,
       0.0156196 , 0.01562563, 0.01530075, 0.01513163, 0.01487461,
       0.01447601, 0.01435962, 0.0141933 , 0.01413464, 0.01400049,
       0.01358944, 0.01324999, 0.01314383, 0.01287846, 0.01264675,
       0.01223567, 0.01202566, 0.01200873, 0.01178434, 0.01170006,
       0.01156842, 0.01125373, 0.01111963, 0.01099734, 0.01077573,
       0.01073484, 0.01047723, 0.01018768, 0.01002638, 0.00983416])

In [157]:
sum(lsa.explained_variance_ratio_)

0.7554491580470581

Create a list of lsa scores to plot on a graph against topic numbers. 

In [153]:
lsa_scores = []
topic_num = []
for i in range(0,100,2):
    lsa = TruncatedSVD(i)
    doc_topic = lsa.fit_transform(doc_word)
    lsa_scores.append(sum(lsa.explained_variance_ratio_))
    topic_num.append(i)
    print(sum(lsa.explained_variance_ratio_))

0
0.0737175414754511
0.15707236423923623
0.208349569019924
0.25207879961485763
0.2910664802972736
0.325395881078035
0.35750925124426447
0.38954187562238574
0.41956325619291984
0.4491691767207802
0.478099385214478
0.5070300510735497
0.5336448505542629
0.5609197905343121
0.5854459063748712
0.6111751567925996
0.6350776174446262
0.6576049025965057
0.6813192801642686
0.7035896957925453
0.7251714760396225
0.7454050623126041
0.7661455343600484
0.7859540339363028
0.8049800417888173
0.8235723878833735
0.8411966624779709
0.8586723899682697
0.8752116812137584
0.8915582668671115
0.9072553048928657
0.921827089986304
0.9359060692123188
0.9492855633367692
0.9620351797467394
0.9738164144005201
0.9845984357156781
0.9935841460576429
1.0000000000000004
1.0
1.0000000000000009
1.0000000000000002
1.0000000000000004
1.0000000000000004
1.0000000000000004
1.0
1.0000000000000004


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
plt.title('SVD Explained variance reaches 1.0\nwhen topics reach number of books')
plt.xlabel('Number of Topics')
plt.ylabel('Explained Variance Ratio')
plt.plot(topic_num, lsa_scores)
plt.axvline(x = 76, color = 'purple', linestyle='--')
plt.text(73,-.12, 76, color ='r')
plt.show()
plt.savefig('../reports/SVD_explained_variance.svg')

In [158]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, tfidf.get_feature_names(), 10)


Topic  0
sir, lady, sister, gentleman, thou, daughter, marry, sea, glance, thee

Topic  1
thou, thee, thy, hath, doth, unto, ship, slave, sea, hast

Topic  2
sister, hath, oblige, society, government, acquaintance, affection, endeavour, shew, liberty

Topic  3
thou, thee, thy, sister, lady, sir, doth, gentleman, unto, thou art

Topic  4
ship, captain, sea, island, boat, shore, king, sail, voyage, majesty

Topic  5
slave, slavery, nigger, negro, dollar, plantation, school, sister, ship, song

Topic  6
king, forest, princess, river, teaching, prince, hath, wealth, kiss, bird

Topic  7
sir, gentleman, sect, hath, king, government, prince, legislative, lady, liberty

Topic  8
car, dollar, tenement, sister, job, train, hath, boss, cent, machine

Topic  9
princess, forest, soldier, sister, king, bridge, river, officer, monk, sergeant

Topic  10
slave, sister, government, sect, slavery, hath, legislative, liberty, remark, bridge

Topic  11
captain, planet, bridge, ship, thou, jack, satellite

In [161]:
Vt = pd.DataFrame(doc_topic.round(5),
             index = book_titles,
             columns = lsa.singular_values_)
# Vt.sort_values('component_5', ascending=False))
Vt
# Vt.sort_values(4.487993476941114, ascending=False)

,3.9344047278131216,1.7613262392246134,1.7306109837673829,1.5334321408577376,1.3345399627876156,1.2280123843889066,1.1970203826036954,1.1860721280886948,1.1386772214639922,1.1135063170286505,...,0.8608856202273406,0.849622389953796,0.8443262039909039,0.8392572752262827,0.8307829235573481,0.8292094185408514,0.8194289378162072,0.8079156317663067,0.8015740213512386,0.7936885479465077
Pride and Prejudice,0.57395,-0.34043,0.52926,0.15774,0.08754,0.03372,-0.04482,-0.20207,0.07631,0.04091,...,0.02144,0.00664,0.03282,-0.00290,0.01799,0.00321,-0.00996,0.02073,0.02134,0.02295
Frankenstein,0.54608,0.12005,0.18711,-0.08543,0.10015,-0.11037,-0.00239,-0.24624,-0.17897,0.06656,...,0.05645,-0.05670,-0.15350,-0.01319,0.04623,-0.05326,0.05899,0.02380,-0.19290,-0.03480
Heart of Darkness,0.44857,0.04917,-0.23630,-0.11737,0.07901,-0.06734,0.04854,-0.17520,-0.06324,0.01637,...,0.09731,0.13512,0.17731,-0.08443,-0.18151,0.29743,-0.13033,0.08037,0.02614,-0.12580
A Modest Proposal,0.21787,0.08189,0.15994,-0.11398,0.06787,0.01821,0.03358,0.18295,0.30248,-0.02346,...,-0.08422,-0.06729,-0.02561,-0.09136,-0.03398,0.00946,0.08480,0.10782,0.01910,0.00992
A Tale of Two Cities,0.61049,-0.07348,-0.07795,0.01840,0.02859,-0.07882,-0.11651,0.13929,-0.07349,0.10239,...,-0.01354,0.07543,0.15049,0.05708,-0.00386,-0.01144,0.20519,-0.06560,0.09587,-0.07025
The Importance of Being Earnest,0.28413,-0.15082,0.08171,0.06575,-0.09861,-0.01520,0.02455,0.11262,-0.14553,-0.16902,...,-0.04622,0.18612,-0.00488,0.11639,-0.01323,-0.08973,-0.00904,0.05452,-0.10808,0.07719
Alice's Adventures in Wonderland,0.35746,-0.12797,-0.17993,0.11235,0.03721,0.21419,0.31276,0.16525,0.07115,-0.12270,...,0.00968,-0.03076,-0.03659,-0.01853,0.01690,0.02292,-0.04603,0.06399,0.06973,-0.15575
Moby Dick; or The Whale,0.47893,0.32033,-0.17476,-0.03369,0.37672,0.06202,-0.18030,-0.11422,-0.03633,-0.20350,...,-0.09825,-0.09079,0.00198,-0.10144,0.00716,0.05826,-0.02366,0.06180,-0.06052,0.11893
Voyage to Jupiter,0.15519,0.05560,-0.03642,-0.14103,-0.09145,-0.23220,0.03698,-0.21058,0.11289,-0.09652,...,0.13473,-0.06486,-0.01522,-0.05597,-0.19827,-0.10491,-0.10755,0.01992,-0.02257,0.06947
The Adventures of Sherlock Holmes,0.63383,-0.15138,-0.11755,-0.00356,-0.08347,-0.21714,-0.15526,0.09330,-0.00478,0.01449,...,-0.01810,-0.08208,0.08712,0.03466,-0.02334,-0.06549,-0.00256,-0.07382,-0.00166,0.05958


## Test Book

Load a book that is not a part of the data trained above.  Transform it with the fit tfidf and the fit svd.
In the example below, I used a Ta-nehisi Coates article.

In [162]:
file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/gutenberg_recommender/data/external/tcoates.txt'
title_of_test = 'tcoates'

In [163]:
#The book_to_compare module opens a text file, cleans it (i.e. removes proper nouns and pronouns)
#then lemmatizes it with spaCy.
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)
print(lemmed_test)

23503
[" only have see there , on the wax hardwood floor of elementary - school auditorium , because be young then , barely    year old , and cable have not yet come to the city , and if have , father would not have believe in . yes , have to have happen like this , like folk wisdom , because when think of that era , do not think of , but of the futile attempt to stay awake and navigate the yawn whiteness of , and remember that there be no vcrs among then , and so would have have to have be there that see , in the auditorium that adjoin the cafeteria , where after the daily serving of tater tot and chocolate milk , a curtain divider be pull back and all the kid storm the stage . and would have be there among , awkwardly uprocking , or worm in place , or stiffly snaking , or back - spinning like a broken rotor , and would have look up and see a kid , slightly old , face , smile to , then move across the floor by pop up alternate heel , glide in reverse , walk on the moon . nothing happe

In [164]:
test_tfidf = tfidf.transform(lemmed_test)
test_lsa = lsa.transform(test_tfidf)
print(test_lsa)

[[ 0.20541817  0.06031908 -0.00708834 -0.12154142 -0.08208239  0.02591016
   0.01476334 -0.02124128 -0.00817966 -0.00084099  0.009015    0.0314411
  -0.01697815 -0.03285514 -0.03071068  0.01219446  0.02538923  0.02342806
   0.00246795 -0.01075574 -0.01124205 -0.02846662 -0.00280394 -0.00431708
  -0.01115019  0.00700326 -0.01806813 -0.00858717  0.01564226  0.02633893
   0.00083904  0.00859782 -0.00471276 -0.00604525  0.00214785  0.00205964
  -0.00662023  0.0127111  -0.02654528  0.01746408 -0.00884127 -0.01479075
   0.00875814  0.00606319 -0.01768953]]


Recompile book info variables just to make sure the information matches the lemmed_text variable.

In [165]:
book_info = get_book_info()
book_nums, book_titles, book_authors = (book_info[1],
                                        book_info[2],
                                        book_info[3])

In [166]:
book_titles.append(title_of_test)

In [167]:
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


Append the test_book transformed into the SVD tops to the array of topics of the training set
to enable cosine similarity and clustering.

In [168]:
doc_for_k = np.concatenate([doc_topic, test_lsa], axis=0)
doc_for_k

array([[ 0.57394815, -0.34042977,  0.52926087, ...,  0.02072572,
         0.02133774,  0.02294634],
       [ 0.54608254,  0.12004916,  0.18710776, ...,  0.02379517,
        -0.19290044, -0.03479932],
       [ 0.44856502,  0.04917394, -0.23630403, ...,  0.08037225,
         0.02613776, -0.12579789],
       ...,
       [ 0.46156767, -0.10134966, -0.28120543, ...,  0.33011112,
        -0.16244496, -0.13333932],
       [ 0.33494548,  0.09084034, -0.17561453, ..., -0.0463769 ,
         0.00839022, -0.01987836],
       [ 0.20541817,  0.06031908, -0.00708834, ...,  0.00875814,
         0.00606319, -0.01768953]])

Check out which topics are most strongly expressed in the test book

In [169]:
test_topics = doc_for_k[-1]
test_topics = np.argsort(test_topics)
print(test_topics)

[ 3  4 13 14 21 38  7 26 44 12 41 20 24 19 40 27  8  2 36 33 32 23 22  9
 30 35 34 18 43 25 31 42 10 15 37  6 28 39 17 16  5 29 11  1  0]


Find most similar books using cosine similarity.

In [114]:
a= cosine_similarity(doc_for_k)

In [115]:
top_cos_sim = np.argsort(a[-1])[:-1]
print(book_titles[top_cos_sim[-1]])
print(book_titles[top_cos_sim[-2]])
print(book_titles[top_cos_sim[-3]])

The Souls of Black Folk
Autographs for Freedom
The Picture of Dorian Gray


In [116]:
num_clusters = 15
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=5) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=5, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [119]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

Heart of Darkness 2
Voyage to Jupiter 2
Metamorphosis 2
The Yellow Wallpaper 2
The Earth Quarter 2
Dracula 2
Walden, and On The Duty Of Civil Disobedience 2
A Study In Scarlet 2
The Jungle 2
The Time Machine 2
My Secret Life, Volumes I. to III. 2
The Hound of the Baskervilles 2
An Occurrence at Owl Creek 2
Anthem 2
Journey Work 2
Micro-Man 2
The Invisible Man 2
The Dunwich Horror 2
tcoates 2


## NMF

In [138]:
from sklearn.decomposition import NMF

#retrain the tfidf vectorizor just to make sure it is fit on the training data.
tfidf= TfidfVectorizer(min_df = .1, max_df = .9, stop_words='english', ngram_range=(1,2))
doc_word = tfidf.fit_transform(lemmed_text_from_dict)

In [139]:
nmf_model = NMF(7)
doc_topic = nmf_model.fit_transform(doc_word)

In [140]:
book_info = get_book_info()
book_nums, book_titles, book_authors = (book_info[1],
                                        book_info[2],
                                        book_info[3])
display_topics(nmf_model, tfidf.get_feature_names(), 20)


Topic  0
sister, father, dear, mother, lady, manner, family, brother, reply, pleasure, feeling, oh, general, letter, gentleman, yes, happy, marry, attention, daughter

Topic  1
power, opinion, nature, subject, government, law, hath, common, sect, society, desire, case, liberty, truth, wealth, public, action, philosopher, accord, individual

Topic  2
boy, oh, girl, dear, yes, mother, lady, laugh, sir, really, hair, gentleman, window, table, till, father, reply, minute, dress, smile

Topic  3
thou, thee, thy, hath, unto, doth, hast, thyself, thou art, thine, dost, thou hast, art, mother, shalt, merry, wilt, aye, ye, minister

Topic  4
ship, sea, captain, boat, island, shore, sail, voyage, kill, king, board, reply, squire, till, cabin, ye, deck, wood, mast, doctor

Topic  5
slave, slavery, master, dollar, tenement, negro, black, white, mother, labor, cent, nigger, plantation, land, freedom, colored, whip, law, school, soul

Topic  6
white, window, black, strange, street, earth, sir, tabl

In [141]:
#bring in the test book, and look at it's topic weights.  It expresses topic 2 most strongly. 
#Topic 2 is heavy in sci fi vocabulary, so it makes sense.

file_loc = '/Users/mbarry/Documents/Coding/ds/metis/projects/gutenberg_recommender/data/external/irobot.txt'
title_of_test = 'irobot'
import book_to_compare
lemmed_test = book_to_compare.new_test_book(file_loc, title_of_test)

test_tf = tfidf.transform(lemmed_test)
test_nmf = nmf_model.transform(test_tf)
test_nmf

42374


array([[0.02528977, 0.02866934, 0.16839095, 0.        , 0.        ,
        0.00604452, 0.16600862]])

In [142]:
book_titles.append(title_of_test)

In [143]:
book_titles

['Pride and Prejudice',
 'Frankenstein',
 'Heart of Darkness',
 'A Modest Proposal',
 'A Tale of Two Cities',
 'The Importance of Being Earnest',
 "Alice's Adventures in Wonderland",
 'Moby Dick; or The Whale',
 'Voyage to Jupiter',
 'The Adventures of Sherlock Holmes',
 'The Souls of Black Folk',
 'Metamorphosis',
 'The Yellow Wallpaper',
 'The Earth Quarter',
 'War and Peace',
 "Grimms' Fairy Tales",
 'Dracula',
 'Jane Eyre',
 'The Picture of Dorian Gray',
 'Walden, and On The Duty Of Civil Disobedience',
 'The Count of Monte Cristo',
 'Leaves of Grass',
 'Autographs for Freedom',
 'Great Expectations',
 'Peter Pan',
 'Dubliners',
 'The Narrative of the Life of Frederick Douglass',
 'Pygmalion',
 'The Romance of Lust',
 'Ulysses',
 'Siddhartha',
 'A Christmas Carol',
 'Treasure Island',
 'Leviathan',
 'Legendary Tales of the Highlands (Volume 1 of 3)',
 'Wuthering Heights',
 'The Essays of Montaigne, Complete',
 'Les Misérables',
 'Songs of Innocence and Songs of Experience',
 'A Stu

In [144]:
doc_for_k = np.concatenate([doc_topic, test_nmf], axis=0)

num_clusters = 10
km = KMeans(n_clusters=num_clusters,random_state=10,n_init=1) # n_init, number of times the K-mean algorithm will run
km.fit(doc_for_k)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
    random_state=10, tol=0.0001, verbose=0)

In [145]:
group_labels = km.labels_
test_group = ''
for group,title in zip(group_labels, book_titles):
    if title == title_of_test:
        test_group = group
for group,title in zip(group_labels, book_titles):
    if group == test_group:
        print(title, group)

A Tale of Two Cities 9
Dracula 9
Jane Eyre 9
The Picture of Dorian Gray 9
Great Expectations 9
Dubliners 9
Ulysses 9
A Christmas Carol 9
irobot 9
